In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy import optimize
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import pymc3 as pm
import arviz as az

In [ ]:
%matplotlib
rows=[]
mdf=pd.read_excel('mf_bli_final.xlsx')
mdf

for idx,mrow in mdf.iterrows():
    df=pd.read_csv(mrow['root']+"/"+mrow['filename']+".csv",skiprows=mrow['skiprow'])
    print(mrow['Peptide'])
    rf=np.array(df[" "+mrow['RF']]-df[" "+mrow['RF']].iloc[mrow['as_start']])[mrow['as_start']:mrow['dis_end']]
    rfc=np.array(df[" "+mrow['RFC']]-df[" "+mrow['RFC']].iloc[mrow['as_start']])[mrow['as_start']:mrow['dis_end']]
    time=np.array(df['X Values']-df['X Values'].iloc[mrow['as_start']])[mrow['as_start']:mrow['dis_end']]
    
    row=pd.Series(rf).to_dict()
    row['Peptide']=mrow['Peptide']
    row['Type']='RF'
    rows.append(row)
    
    row=pd.Series(rfc).to_dict()
    row['Peptide']=mrow['Peptide']
    row['Type']='RFC'
    rows.append(row)
    
    row=pd.Series(time).to_dict()
    row['Peptide']=mrow['Peptide']
    row['Type']='time'
    row['Phase']='full'
    rows.append(row)
    
    row=pd.Series(time[mrow['as_start']:mrow['switch']]).to_dict()
    row['Peptide']=mrow['Peptide']
    row['Type']='time'
    row['Phase']='as'
    rows.append(row)
    
    row=pd.Series(time[mrow['switch']-mrow['dis_end']:]-time[mrow['switch']-mrow['dis_end']]).to_dict()
    row['Peptide']=mrow['Peptide']
    row['Type']='time'
    row['Phase']='dis'
    rows.append(row)

    
    for c in ['conc1','conc2','conc3','conc4','conc5','conc6']:
        conc=np.array(df[" "+mrow[c]]-df[" "+mrow[c]].iloc[mrow['as_start']])[mrow['as_start']:mrow['dis_end']]-rf
        row=pd.Series(conc).to_dict()
        row['Peptide']=mrow['Peptide']
        row['Conc']=c
        row['Type']='pept'
        row['Phase']='full'
        rows.append(row)
        
        conc=np.array(df[" "+mrow[c]]-df[" "+mrow[c]].iloc[mrow['as_start']])[mrow['as_start']:mrow['switch']]-rf[mrow['as_start']:mrow['switch']]
        row=pd.Series(conc).to_dict()
        row['Peptide']=mrow['Peptide']
        row['Conc']=c
        row['Type']='pept'
        row['Phase']='as'
        rows.append(row)
        
        conc=np.array(df[" "+mrow[c]]-df[" "+mrow[c]].iloc[mrow['as_start']])[mrow['switch']:mrow['dis_end']]-rf[mrow['switch']-mrow['dis_end']:]
        row=pd.Series(conc).to_dict()
        row['Peptide']=mrow['Peptide']
        row['Conc']=c
        row['Type']='pept'
        row['Phase']='dis'
        rows.append(row)
        
    for c in ['conc1_C','conc2_C','conc3_C','conc4_C','conc5_C','conc6_C']:
        conc=np.array(df[" "+mrow[c]]-df[" "+mrow[c]].iloc[mrow['as_start']])[mrow['as_start']:mrow['dis_end']]-rfc
        row=pd.Series(conc).to_dict()
        row['Peptide']=mrow['Peptide']
        row['Conc']=c
        row['Type']='ctrl'
        rows.append(row)
    plt.plot(time,conc)
tf=pd.DataFrame(rows)
tf

In [49]:
resrows=[]
peptides=['EZH2_172-211_INVERSE',
          'EZH2_172-211',

         ]
concs=['conc1','conc2','conc3','conc4']
str2conc={('EZH2_172-211_INVERSE','conc1'):21.2E-6,
          ('EZH2_172-211_INVERSE','conc2'):2.12E-6,
          ('EZH2_172-211_INVERSE','conc3'):2.12E-7,
          ('EZH2_172-211_INVERSE','conc4'):2.12E-8,
          ('EZH2_172-211','conc1'):100E-9,
          ('EZH2_172-211','conc2'):33E-9,
          ('EZH2_172-211','conc3'):11E-9,
          ('EZH2_172-211','conc4'):3.7E-9,
         }

for peptide in peptides:
    for conc in concs:
        print (peptide,conc)
        atime=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='time')&(tf.Phase=='as')])[0,:-4],dtype='float')
        atime=atime[~np.isnan(atime)]
        aval=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='pept')&(tf.Phase=='as')&(tf.Conc==conc)])[0,:-4],dtype='float')
        aval=aval[~np.isnan(aval)]
        x=np.array(atime)
        y=aval
        dtime=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='time')&(tf.Phase=='dis')])[0,:-4],dtype='float')
        dtime=dtime[~np.isnan(dtime)]
        dval=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='pept')&(tf.Phase=='dis')&(tf.Conc==conc)])[0,:-4],dtype='float')
        dval=dval[~np.isnan(dval)]

        xd=np.array(dtime)
        yd=dval
          
        c=str2conc[(peptide,conc)]
        print(c)
        with pm.Model() as model2:
            KD=pm.Uniform('KD',1E-15,1E-3)
            A=pm.Uniform('A',1E-6,10)
            eps=pm.Uniform('eps',1E-6,1)
            l0=pm.TruncatedNormal('l0',mu=np.mean(dval[0:5]),sigma=np.std(dval[0:5])/10,lower=0)
            
            kr1=pm.Uniform('kr1',1E-20,10)
            k1=pm.Deterministic('k1',kr1/KD)
            pred = pm.Normal('pred', mu=A*c*(1-pm.math.exp(-1.0*x*(k1*c+kr1)))/(KD+c), sd=eps, observed=np.array(y))
            pred2 = pm.Normal('pred2', mu=l0*(pm.math.exp(-1.0*xd*kr1)), sd=eps, observed=np.array(yd))

        with model2:
            trace2 = pm.sampling.sample(tune=10000,init='advi')
            resrow={}
            resrow['Peptide']=peptide
            resrow['Conc']=conc
            resrow['Phase']='full'
            resrow['MCMC']='trace'
            resrow['Model']='1exp'
            resrow['Pymodel']=model2
            resrow['Data']=trace2
            resrow['X']=x
            resrow['Y']=y
            resrows.append(resrow)

        with model2:
            ppc2 = pm.sample_posterior_predictive(trace2, samples=1000 ,model=model2)
            resrow={}
            resrow['Peptide']=peptide
            resrow['Conc']=conc
            resrow['Phase']='full'
            resrow['MCMC']='ppc'
            resrow['Model']='1exp'
            resrow['Pymodel']=model2
            resrow['Data']=ppc2
            resrows.append(resrow)
        
resdf=pd.DataFrame(resrows)

EZH2_172-211_INVERSE conc1
2.12e-05


Auto-assigning NUTS sampler...
Initializing NUTS using advi...


Convergence achieved at 16600
Interrupted at 16,599 [8%]: Average Loss = 1.3137e+06
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [kr1, l0, eps, A, KD]


Sampling 4 chains for 10_000 tune and 1_000 draw iterations (40_000 + 4_000 draws total) took 1352 seconds.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.
/home/gergely/anaconda3/lib/python3.7/site-packages/pymc3/sampling.py:1618: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "


EZH2_172-211_INVERSE conc2
2.12e-06


...

Sampling 4 chains for 10_000 tune and 1_000 draw iterations (40_000 + 4_000 draws total) took 145 seconds.
The number of effective samples is smaller than 25% for some parameters.
/home/gergely/anaconda3/lib/python3.7/site-packages/pymc3/sampling.py:1618: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "


EZH2_172-211 conc4
3.7e-09


Auto-assigning NUTS sampler...
Initializing NUTS using advi...


Convergence achieved at 29400
Interrupted at 29,399 [14%]: Average Loss = 2.3708e+08
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [kr1, l0, eps, A, KD]


Sampling 4 chains for 10_000 tune and 1_000 draw iterations (40_000 + 4_000 draws total) took 180 seconds.
The number of effective samples is smaller than 10% for some parameters.
/home/gergely/anaconda3/lib/python3.7/site-packages/pymc3/sampling.py:1618: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "


In [60]:
rrows=[]
peptides=['EZH2_172-211','EZH2_172-211_INVERSE',
          #"SUZ12_573-597","EED_128-152","JARID2_170-194"
         ]
concs=['conc1','conc2','conc3','conc4']

for peptide in peptides:
    for conc in concs:
        trace2=resdf[(resdf.Peptide==peptide)&(resdf.Conc==conc)&(resdf.Phase=='full')&(resdf.MCMC=='trace')&(resdf.Model=='1exp')]['Data']
        trace2=trace2.iloc[0]
        row={}
        row['Peptide']=peptide
        row['Concentration']=conc
        row['Phase']='full'
        row['Model']='1exp'
        row['A']='%4.1E±%4.1E' % (np.mean(trace2['A']),np.std(trace2['A']))
        row['KD']='%4.1E±%4.1E' % (np.mean(trace2['KD']),np.std(trace2['KD']))
        row['l0']='%4.1E±%4.1E' % (np.mean(trace2['l0']),np.std(trace2['l0']))
        row['kr1']='%4.1E±%4.1E' % (np.mean(trace2['l0']),np.std(trace2['l0']))
        row['k1']='%4.1E±%4.1E' % (np.mean(trace2['k1']),np.std(trace2['k1']))
        row['eps']='%4.1E±%4.1E' % (np.mean(trace2['eps']),np.std(trace2['eps']))

        rrows.append(row)


rdf=pd.DataFrame(rrows)
rdf.to_excel('BLI_table_sd.xlsx')
rdf

,Peptide,Concentration,Phase,Model,A,KD,l0,kr1,k1,eps
0,EZH2_172-211,conc1,full,1exp,2.3E-01±1.1E-04,7.0E-10±6.7E-12,2.3E-01±7.5E-05,2.3E-01±7.5E-05,5.0E+04±8.4E+01,3.5E-03±2.0E-05
1,EZH2_172-211,conc2,full,1exp,1.1E-01±1.8E-04,3.5E-10±6.5E-12,1.0E-01±5.9E-05,1.0E-01±5.9E-05,8.8E+04±3.3E+02,2.9E-03±1.7E-05
2,EZH2_172-211,conc3,full,1exp,5.5E-02±5.5E-04,6.1E-10±1.4E-11,4.0E-02±4.5E-05,4.0E-02±4.5E-05,1.3E+05±2.2E+03,2.9E-03±1.8E-05
3,EZH2_172-211,conc4,full,1exp,1.8E-01±2.1E-01,6.7E-04±2.4E-04,3.9E-07±3.8E-07,3.9E-07±3.8E-07,1.1E+04±2.5E+04,4.3E-03±2.7E-05
4,EZH2_172-211_INVERSE,conc1,full,1exp,1.3E-01±1.2E-03,1.9E-09±1.3E-09,7.0E-02±3.1E-05,7.0E-02±3.1E-05,4.5E+01±5.9E-01,3.0E-03±1.9E-05
5,EZH2_172-211_INVERSE,conc2,full,1exp,3.6E-02±1.8E-04,3.1E-11±2.1E-11,3.9E-02±3.7E-05,3.9E-02±3.7E-05,1.6E+03±2.0E+01,3.8E-03±2.3E-05
6,EZH2_172-211_INVERSE,conc3,full,1exp,2.1E-02±7.2E-04,1.0E-11±6.7E-12,2.4E-02±4.5E-05,2.4E-02±4.5E-05,7.5E+03±4.2E+02,4.3E-03±2.7E-05
7,EZH2_172-211_INVERSE,conc4,full,1exp,8.0E-03±3.6E-04,2.8E-12±2.1E-12,9.7E-03±3.4E-05,9.7E-03±3.4E-05,9.0E+04±7.2E+03,3.3E-03±2.0E-05


In [65]:
%matplotlib
peptides=[
        'EZH2_172-211', 'EZH2_172-211_INVERSE',
         ]


astime=900
for peptide in peptides:
    plt.rcParams.update({'font.size': 8})
    fig,ax=plt.subplots(4,2,dpi=300,constrained_layout=True,sharex=True,sharey=True)
    for idx2,expo in enumerate(['1exp',
                                #'2exp'
                               ]):
        for idx,conc in enumerate([
                                    'conc1',
                                   'conc2','conc3','conc4',

                                    ]):
            atime=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='time')&(tf.Phase=='as')])[0,:-4],dtype='float')
            atime=atime[~np.isnan(atime)]

            aval=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='pept')&(tf.Phase=='as')&(tf.Conc==conc)])[0,:-4],dtype='float')
            aval=aval[~np.isnan(aval)]

            dtime=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='time')&(tf.Phase=='dis')])[0,:-4],dtype='float')
            dtime=dtime[~np.isnan(dtime)]

            dval=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='pept')&(tf.Phase=='dis')&(tf.Conc==conc)])[0,:-4],dtype='float')
            dval=dval[~np.isnan(dval)]

            x=np.concatenate([atime,dtime+astime])
            y=np.concatenate([aval,dval])
            ppc_as=resdf[(resdf.Peptide==peptide)&(resdf.Conc==conc)&(resdf.Phase=='full')&(resdf.MCMC=='ppc')&(resdf.Model==expo)]['Data']
            ppc_dis=resdf[(resdf.Peptide==peptide)&(resdf.Conc==conc)&(resdf.Phase=='full')&(resdf.MCMC=='ppc')&(resdf.Model==expo)]['Data']
            ppc=np.concatenate([np.array(ppc_as.iloc[0]['pred']),np.array(ppc_dis.iloc[0]['pred2'])],axis=1)
            ax[idx,idx2].plot(x,y-np.mean(ppc, axis=0), color='gray')
            ax[idx,idx2].plot(x,np.zeros(len(x)), color='r')

            ax[idx,0].set_ylabel(r'$\Delta\lambda\ (nm)$')
            ax[idx,idx2].axvline(x=900,linestyle='--')
    ax[idx,0].set_xlabel(r'$Time\ (s)$')
    ax[idx,1].set_xlabel(r'$Time\ (s)$')
    
    plt.savefig('Langmuir_residual_%s.png' %(peptide))

Using matplotlib backend: Qt5Agg


In [ ]:
%matplotlib
peptides=[
          'EZH2_172-211','EZH2_172-211_INVERSE',
         ]

ccol_C={'conc1_C':cmap(0),
      'conc2_C':cmap(0.33),
      'conc3_C':cmap(0.66),
      'conc4_C':cmap(1.0),

     }

cdict_C={('EZH2_172-211','conc1_C'):r'$100\ nM$',
       ('EZH2_172-211','conc2_C'):r'$33\ nM$',
       ('EZH2_172-211','conc3_C'):r'$11\ nM$',
       ('EZH2_172-211','conc4_C'):r'$3.7\ nM$',
       ('EZH2_172-211','conc5_C'):r'$1.2\ nM$',
       ('EZH2_172-211_INVERSE','conc1_C'):r'$210\ μM$',
       ('EZH2_172-211_INVERSE','conc2_C'):r'$21\ μM$',
       ('EZH2_172-211_INVERSE','conc3_C'):r'$2.1\ μM$',
       ('EZH2_172-211_INVERSE','conc4_C'):r'$210\ nM$',
       ('EZH2_172-211_INVERSE','conc5_C'):r'$21\ nM$',
      }


astime=900

for peptide in peptides:
    plt.rcParams.update({'font.size': 16})
    fig,ax=plt.subplots(dpi=300,constrained_layout=True)
    clst=[]
    for idx,conc in enumerate(['conc1_C','conc2_C','conc3_C','conc4_C']):
        x=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='time')&(tf.Phase=='full')])[0,:-4],dtype='float')
        y=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='ctrl')&(tf.Conc==conc)])[0,:-4],dtype='float')
        ax.plot(x[::20],y[::20],c=ccol_C[conc])
        ax.set_xlabel(r'$Time\ (s)$')
        ax.set_ylabel(r'$\Delta\lambda\ (nm)$')
        clst.append(cdict_C[(peptide,conc)])
    plt.rcParams.update({'font.size': 12})
    plt.legend(clst)
    plt.savefig('Langmuir_ctrl_%s.png' %(peptide))

In [64]:
%matplotlib
from matplotlib.ticker import FormatStrFormatter
peptides=[
          'EZH2_172-211', 'EZH2_172-211_INVERSE',

         ]

ccol_C={'conc1_C':cmap(0),
      'conc2_C':cmap(0.33),
      'conc3_C':cmap(0.66),
      'conc4_C':cmap(1.0),
     }

cdict_C={('EZH2_172-211','conc1'):r'$100\ nM\ survivin$',
       ('EZH2_172-211','conc2'):r'$33\ nM\ survivin$',
       ('EZH2_172-211','conc3'):r'$11\ nM\ survivin$',
       ('EZH2_172-211','conc4'):r'$3.7\ nM\ survivin$',
       ('EZH2_172-211','conc5'):r'$1.2\ nM\ survivin$',
       ('EZH2_172-211_INVERSE','conc1'):r'$21.2\ μM\ EZH2_{172-211}$',
       ('EZH2_172-211_INVERSE','conc2'):r'$2.12\ μM\ EZH2_{172-211}$',
       ('EZH2_172-211_INVERSE','conc3'):r'$212\ nM\ EZH2_{172-211}$',
       ('EZH2_172-211_INVERSE','conc4'):r'$21.2\ nM\ EZH2_{172-211}$',
      }
cdict={('EZH2_172-211','conc1'):r'$100\ nM\ survivin$',
       ('EZH2_172-211','conc2'):r'$33\ nM\ survivin$',
       ('EZH2_172-211','conc3'):r'$11\ nM\ survivin$',
       ('EZH2_172-211','conc4'):r'$3.7\ nM\ survivin$',
       ('EZH2_172-211','conc5'):r'$1.2\ nM\ survivin$',
       ('EZH2_172-211_INVERSE','conc1'):r'$21.2\ μM\ EZH2_{172-211}$',
       ('EZH2_172-211_INVERSE','conc2'):r'$2.12\ μM\ EZH2_{172-211}$',
       ('EZH2_172-211_INVERSE','conc3'):r'$212\ nM\ EZH2_{172-211}$',
       ('EZH2_172-211_INVERSE','conc4'):r'$21.2\ nM\ EZH2_{172-211}$',
      }

cmap = cm.get_cmap('cool')
ccol={'conc1':cmap(0),
      'conc2':cmap(0.33),
      'conc3':cmap(0.66),
      'conc4':cmap(1.0),
     }


expo='1exp'
astime=900

for peptide in peptides:
    plt.rcParams.update({'font.size': 16})
    fig,ax=plt.subplots(nrows=1,ncols=2,figsize=(15,5), dpi=300,constrained_layout=True, sharey=True)
    clst=[]
    for idx,conc in enumerate([
                                'conc1',
                               'conc2','conc3','conc4',
                                #'conc5'
                                ]):
        atime=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='time')&(tf.Phase=='as')])[0,:-4],dtype='float')
        atime=atime[~np.isnan(atime)]

        aval=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='pept')&(tf.Phase=='as')&(tf.Conc==conc)])[0,:-4],dtype='float')
        aval=aval[~np.isnan(aval)]
        
        dtime=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='time')&(tf.Phase=='dis')])[0,:-4],dtype='float')
        dtime=dtime[~np.isnan(dtime)]
        
        dval=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='pept')&(tf.Phase=='dis')&(tf.Conc==conc)])[0,:-4],dtype='float')
        dval=dval[~np.isnan(dval)]

        x=np.concatenate([atime,dtime+astime])
        y=np.concatenate([aval,dval])
        ppc_as=resdf[(resdf.Peptide==peptide)&(resdf.Conc==conc)&(resdf.Phase=='full')&(resdf.MCMC=='ppc')&(resdf.Model==expo)]['Data']
        ppc_dis=resdf[(resdf.Peptide==peptide)&(resdf.Conc==conc)&(resdf.Phase=='full')&(resdf.MCMC=='ppc')&(resdf.Model==expo)]['Data']
        ppc=np.concatenate([np.array(ppc_as.iloc[0]['pred']),np.array(ppc_dis.iloc[0]['pred2'])],axis=1)
        ax[0].plot(x[::20],y[::20], color='k',label='_nolegend_',alpha=0.5)
        ax[0].plot(x,np.mean(ppc, axis=0),c=ccol[conc])
        ax[0].set_xlabel(r'$Time\ (s)$')
        ax[0].set_ylabel(r'$\Delta\lambda\ (nm)$')
        clst.append(cdict[(peptide,conc)])
        

    
    plt.rcParams.update({'font.size': 16})

    for idx,conc in enumerate([
                                'conc1_C',
                               'conc2_C','conc3_C','conc4_C',

                                ]):

        x=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='time')&(tf.Phase=='full')])[0,:-4],dtype='float')
        y=np.array(np.array(tf[(tf.Peptide==peptide)&(tf.Type=='ctrl')&(tf.Conc==conc)])[0,:-4],dtype='float')
        ax[1].plot(x[::20],y[::20],c=ccol_C[conc])
        ax[1].set_xlabel(r'$Time\ (s)$')
        ax[1].set_ylabel(r'$\Delta\lambda\ (nm)$')
        plt.tick_params('y', labelleft=True)
       
    plt.rcParams.update({'font.size': 12})
    plt.legend(clst)
    ax[1].axvline(x=900,linestyle='--',c='k')
    ax[0].axvline(x=900,linestyle='--',c='k')
    
    ax[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[1].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    plt.savefig('Langmuir_ppc_ctrl_%s_%s.png' %(peptide,expo))


Using matplotlib backend: Qt5Agg


In [74]:
peptide='EZH2_172-211_INVERSE'
#peptide='EZH2_172-211'

conc='conc4'
trace=resdf[(resdf.Peptide==peptide)&(resdf.Conc==conc)&(resdf.Phase=='full')&(resdf.MCMC=='trace')&(resdf.Model=='1exp')]['Data'].iloc[0]
pm.traceplot(trace)

/home/gergely/anaconda3/lib/python3.7/site-packages/arviz/data/io_pymc3.py:91: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


array([[<AxesSubplot:title={'center':'KD'}>,
        <AxesSubplot:title={'center':'KD'}>],
       [<AxesSubplot:title={'center':'A'}>,
        <AxesSubplot:title={'center':'A'}>],
       [<AxesSubplot:title={'center':'eps'}>,
        <AxesSubplot:title={'center':'eps'}>],
       [<AxesSubplot:title={'center':'l0'}>,
        <AxesSubplot:title={'center':'l0'}>],
       [<AxesSubplot:title={'center':'kr1'}>,
        <AxesSubplot:title={'center':'kr1'}>],
       [<AxesSubplot:title={'center':'k1'}>,
        <AxesSubplot:title={'center':'k1'}>]], dtype=object)